
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
#!pip install detectron2 -f  https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.8/index.html
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

In [1]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:
import torch
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

True

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive 
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

Mounted at /content/drive


In [2]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = '/content/drive/My Drive/SFU_CMPT762_ComputerVision/Assignment 3'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
#os.removedirs(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
OUTPUT_DIR2 = '{}/output2'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR2, exist_ok=True)

In [ ]:
#os.removedirs(OUTPUT_DIR)

## Part 1: Object Detection

### Data Loader

In [4]:
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''
from os import listdir
# process custom dataset
def get_detection_data(set_name):
   data_dirs = '{}/data'.format(BASE_DIR)
   dataset = []

   if set_name == "train":
      json_file = os.path.join(data_dirs, "train.json")
      with open(json_file) as f:
            #print(json_file)
            imgs_anns = json.load(f)

      dataset = []
      for idx, ann in enumerate(imgs_anns):
            record = {}

            #detected obj
            obj = {
                "bbox": ann["bbox"],
                "bbox_mode": BoxMode.XYWH_ABS,
                "segmentation": ann["segmentation"] ,
                "category_id": 0 ,
            }  

            filename = os.path.join(data_dirs, set_name, ann["file_name"])

            count = 0
            while count < len(dataset):
                if dataset[count]["file_name"] == filename:
                    dataset[count]["annotations"].append(obj)
                    break
                else:
                    count += 1 

            if count == len(dataset):
                width,height = Image.open(filename).size
                image_id = ann["image_id"]
                record["file_name"] = filename
                record["image_id"] = image_id
                record["height"] = height
                record["width"] = width

                objs = []
                objs.append(obj)
                record["annotations"] = objs
                dataset.append(record)

   if set_name == "test":
      #get text files list
       for  idx , f in enumerate(listdir(os.path.join(data_dirs, set_name))):
            record = {}

            filename = os.path.join(data_dirs,'test',f)
            width,height = Image.open(filename).size

            record["file_name"] = filename
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width

            dataset.append(record)
      
   return dataset

In [5]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
for d in ["train", "test"]:
    DatasetCatalog.register("data_detection_" + d, lambda d=d: get_detection_data( d ))
    MetadataCatalog.get("data_detection_" + d).set(thing_classes=["plane"])
plane_train_meta = MetadataCatalog.get("data_detection_train")
plane_test_meta = MetadataCatalog.get("data_detection_test")


In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''
import time
stime = time.time()
dataset_dicts = get_detection_data("train")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=plane_train_meta, scale=0.1)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

print("execute time " , time.time() - stime)

### Set Configs

In [26]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)


cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
#c   350- 0.503
#"COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml" --800 - 0.488 |     700-54.5%
#COCO-Detection/retinanet_R_101_FPN_3x.yaml --600 0.397
#COCO-Detection/rpn_R_50_FPN_1x.yaml

cfg.DATASETS.TRAIN = ("data_detection_train")
cfg.DATASETS.TEST = ()

cfg.DATALOADER.NUM_WORKERS = 2

#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.STEPS = [] 

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512



### Training

In [25]:
# customer DataLoader and define own trainer
import detectron2.data.transforms as T
from detectron2.data import build_detection_train_loader
from detectron2.data import DatasetMapper   # the default mapper

class MyTrainer(DefaultTrainer):
  @classmethod
  def build_train_loader(cls, cfg):
      mapper =  DatasetMapper(cfg, is_train=True, augmentations=[
        #T.resize((900,900)),
        T.ResizeShortestEdge(1200,max_size =2000),  # orignal image too big, lead to cuda out of memory 
        T.RandomCrop("relative",(0.7,0.7) ),
        #T.ResizeShortestEdge(800,max_size =1500),
        #T.RandomBrightness(0.3,1.5),
        T.RandomFlip(),
        #T.RandomRotation([0,360])
        ])
      
      return build_detection_train_loader(cfg, mapper= mapper)

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
#trainer = DefaultTrainer(cfg) 
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


### Evaluation and Visualization

In [22]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor_detector = DefaultPredictor(cfg)




In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
from detectron2.utils.visualizer import ColorMode
# load test data
dataset_dicts_test = get_detection_data("test")

for d in [dataset_dicts_test[10]]:    
    im = cv2.imread(d["file_name"])
    outputs = predictor_detector(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata= plane_test_meta, 
                   scale=0.4, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])
    # for b in outputs['instances'].pred_boxes:


In [ ]:
'''
# Use COCOEvaluator and build_detection_test_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data_detection_train", cfg, False, output_dir="./output/")
#evaluator = COCOEvaluator("data_detection_train", ("bbox", "segm"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(trainer.model, val_loader, evaluator))



### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
'''
# try other method

# use pretrained model

#data augmentaton
#before augmentation  IOU around 50%

#Data processing



'\n# Bring any changes and updates regarding the improvement in here\n'

## Part 2: Semantic Segmentation

### Data Loader

In [ ]:
from detectron2.utils.visualizer import GenericMask


In [48]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''
from  detectron2.utils.visualizer import GenericMask
import matplotlib.pyplot as plt
def get_instance_sample(data, idx, orig_img=None):
  # get image from data
  image_path = data['file_name']
  im_h, im_w= data['height'], data['width']

  # get segmentation data
  segmenation = data['annotations'][idx]['segmentation']
  b_x, b_y, b_w, b_h = data['annotations'][idx]['bbox']

  #print(segmenation)
  # convert segmentaton data to binary masks

  #print("height, width",im_h, im_w)
  full_mask = GenericMask(segmenation, im_h, im_w)
  #print(full_mask.mask)
  obj_mask = np.array(full_mask.mask)
  #print("---",(obj_mask))

  # crop image according to the size of target  box
  #full_im = Image.open(image_path)  #[:,:,::-1]
  obj_img = orig_img[round(b_y):round(b_y+b_h), round(b_x):round(b_x+b_w)]
  #obj_img = np.array(obj_img)

  obj_mask = obj_mask[round(b_y):round(b_y+b_h), round(b_x):round(b_x+b_w)]
  
  #plt.imshow(obj_mask)
  #obj_img.show()
  #cv2_imshow(np.array(obj_img)[:,:,::-1])
  
  return obj_img, obj_mask

In [49]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''
class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
      self.transforms = transforms.Compose([         
          # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
          transforms.ToTensor(),
          # normalization 
          #transforms.Normalize((.5,.5,.5),(.5,.5,.5))  
      ])

      self.set_name = set_name
      #self.data = data_list
      #---------------------------back to full when train----
      self.data = data_list   #[:50]
      self.instance_map = []
      self.img_dict = {}
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])          
        self.img_dict[d['file_name']] = cv2.imread(d['file_name'])
      print("total_image_number", len(self.data))
      #store images data, avoid reading drive frequentl
      

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
      return len(self.instance_map)

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
        #print("after trans-----", torch.sum(img))
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    #print(idx, self.instance_map[idx])

    idx = self.instance_map[idx]
    data = self.data[idx[0]]
    
    # create self.img_dict save images read from drive
    #check if image data in dict, if not then append, reduce frequency of reading imagre from drive
    # orig_img= []
    image_path = data['file_name']
    orig_img = self.img_dict[image_path]      
    # get instance from data
    img, mask =  get_instance_sample(data, idx[1], orig_img)

    #resize image and mask to fix size 
    img =  cv2.resize(img, (128,128))
    # use Image resize, then convert back to np.array
    #mask = mask.resize((128,128))
    mask = np.array(Image.fromarray(mask).resize((128,128)))

    # convert numpy to tensor
    img, mask = self.numpy_to_tensor( np.array(img), mask)
    #print("Aftter ----input image =mask== name ---", img.size(), mask.size(), image_path)
    return img, mask

def get_plane_dataset(set_name='train', batch_size=2):
    my_data_list = DatasetCatalog.get("data_detection_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

## load data

### Network

In [20]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=True)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)  
             )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            conv(out_ch, out_ch),
            nn.MaxPool2d(2)
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv1 = conv(in_ch, out_ch)
        self.conv2 = conv(out_ch, out_ch)

    def forward(self, x):
        y = self.up(x)
        y = self.conv1(y)
        y = self.conv2(y)
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # 1. increase to 4 layers
        # 2. skip connection
        # Encoder
        self.input_conv = conv(3, 16)
        self.down1 = down(16,32)
        self.down2 = down(32,64)
        self.down3 = down(64, 128)
        self.down4 = down(128, 256)
        

        # Decoder
        self.up1 = up(256, 128)
        self.up2 = up(128, 64)
        self.up3 = up(64, 32)
        self.up4 = up(32, 16)

        self.output_conv = conv(16, 1, False) # ReLu activation is removed to keep the logits for the loss function
        

    def forward(self, input):
      y = self.input_conv(input) #3->16
      #pre1 = y
      y = self.down1(y) # 16->32
     # pre2 = y
      y = self.down2(y) #32->64
      #pre3 = y   
      y = self.down3(y)  #64->128
      #pre4 =y
      y = self.down4(y)  #128->256

      y = self.up1(y)    #256->128
      y = self.up2(y)
      y = self.up3(y)
      y = self.up4(y)

      output = self.output_conv(y)
      return output

### Training

In [ ]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''


batch_size = 4
# model = MyModel() # initialize the model
# model = model.cuda() # move the model to GPU
print("start loader")
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
print("finish--loader")



start loader
total_image_number 198
finish--loader


In [ ]:
# Set the hyperparameters
num_epochs = 5
batch_size = 4
learning_rate = 0.001
weight_decay = 1e-4

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
crit = nn.BCEWithLogitsLoss() # Define the loss function
#crit = nn.CrossEntropyLoss() # Define the loss function
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))

#++++++======================== try adm optimizer and different learning_rate
#optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as SGD
optim = torch.optim.Adam(model.parameters(), lr= 0.0002)

# start the training procedure
for epoch in range(num_epochs):
  total_loss = 0
  for (img, mask) in tqdm(loader):
    #print( np.sum(np.array(img)<0))
    img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
    mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = False)  

    mask = torch.unsqueeze(mask, 1)  # (4,128,128) -> (4,1,128,128)
    pred = model(img)
    loss = crit(pred, mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))

'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))


### Evaluation and Visualization

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
batch_size = 8
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers
#loader, dataset = get_plane_dataset('train', batch_size)
#loader  already got

threshold = 0.5
total_iou = 0
for (img, mask) in tqdm(loader):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cuda()
    mask = torch.unsqueeze(mask,1)
    pred = model(img)
    #conver values to [0 , 1]
    pred = torch.sigmoid(pred)
    pre_np = torch.squeeze(pred,1).cpu().numpy()
    m_np = torch.squeeze(mask,1).cpu().numpy()

    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''
    b_size = pre_np.shape[0]
    pred_m = pred > threshold
    i_mask = np.sum(pre_np*m_np, axis= (1,2))
    u_mask = np.sum((pre_np + m_np - pre_np*m_np), axis= (1,2))
    #print(i_mask.shape)
    iou = i_mask/u_mask
    #print("iou---",iou)
    avg_iou = np.sum(iou)/b_size

    total_iou += avg_iou

print("\n #images: {}, Mean IoU: {}".format("plane", total_iou/len(loader)))


In [51]:
# test_im_path = os.path.join(BASE_DIR , "data/train/P1741.png")
# test_img = Image.open(test_im_path)
# print(np.array(test_img).shape)
model = MyModel() # initialize the model
model = model.cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))

<All keys matched successfully>

In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''
#  "test " get_plane_dataset
# crop plane from 3 sample in test_data
threshold = 0.5
test_im_path = os.path.join(BASE_DIR , "data/test/P0257.png")
test_img = Image.open(test_im_path)
test_img.show()
crop1 = np.array(test_img.crop((688,443,788,523)).resize((128,128)))
crop2 = np.array(test_img.crop((390,823,440,873)).resize((128,128)))
crop3 = np.array(test_img.crop((300,1254,355,1310)).resize((128,128)))

transform = transforms.Compose([ transforms.ToTensor()])

#dataset = transform(np.array[crop1, crop2, crop3])
samples = [ crop1, crop2, crop3 ]
model = model.eval()
for img in samples:
  with torch.no_grad():
    input = transform(img)
    input = input.view(1,3,128,128)
    input = input.cuda()
    pred = model(input)
    #print(pred.size())
    plt.subplot(1,2,1)
    pred_mask = np.array(pred.cpu().view(128,128))>threshold
    plt.imshow(pred_mask)
    plt.subplot(1,2,2)
    pred_mask = np.reshape(pred_mask,(128,128,1))
    pred_mask = np.concatenate((pred_mask,pred_mask,pred_mask),2)
    plt.imshow(np.multiply(img,pred_mask))
    plt.show()
    # cv2_imshow( pred.cpu().reshape(128,128).numpy() )
    # cv2_imshow( pred.cpu().reshape(128,128).numpy()* im.numpy())


## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [54]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''

def get_prediction_mask(data):  # data is a record of sample image with annotations
   # get information from data
   width = data["width"]
   height = data["height"]
   annotations = []
   if 'annotations' in data.keys():
      annotations = data['annotations']

   #default predictor tak BRG data for input
   img = cv2.imread(data["file_name"])
   #im = Image.open(data["file_name"])
   #Use predictor from Part1 to detect bboxes
   detoctor_data  =  predictor_detector(img)
   instances = detoctor_data["instances"]
   #print(instances)
   # get bbox - list[tensor]
   boxes = instances.pred_boxes

   #initialize image_mask
   pre_mask_all = np.zeros((height,width))
   gt_mask = np.zeros((height,width))
   
   #print("dectected box:",len(boxes))

   for idx,box in enumerate(boxes):
        #location_box, size_box 
        location = box.cpu().numpy().astype(int)
        x1,y1,x2,y2 = list(location)  
        b_w = x2 - x1
        b_h = y2 - y1
        #print("box size", b_w, b_h)
        #print("box location", x1,y1,x2,y2)
        
        # get input by cropping from test image
        crop_im = img[y1:y2, x1:x2,:]
        crop_im = np.array(cv2.resize(crop_im,(128,128)))

        transform = transforms.Compose([ transforms.ToTensor(),transforms.Normalize((0,0,0),(1,1,1))])    
        crop_input = transform(crop_im).view(1,3,128,128)
        
        #pridict mask
        with torch.no_grad():
          pred_mask = model(crop_input.cuda())

        #draw the pred_mask on orignal image  using  location and size of box
        #conver to [0 , 1]
        pred_mask = torch.sigmoid(pred_mask)
        #convert to numpy ndarray and binarize
        pred_mask = np.array(pred_mask.view(128,128).cpu())
        recon_mask = np.array(Image.fromarray(pred_mask).resize((b_w,b_h)))>threshold
        #recon_mask = pred_mask.resize((b_h,b_w))
        # plt.imshow(recon_mask)
        # plt.show()

        #replace detected box on the original image, seperate value mask for different instance
        pre_mask_all[y1:y2,x1:x2] = recon_mask * (idx+1)
        
   # get ground true segementation 

   #gt_obj, gt_mask = get_instance_sample(data, idx, im)
   #orginize segementaion list , then use "Genemask" to convert binary mask
   segmentation_all = []
   for ann in annotations:
     segmentation_all.append(ann['segmentation'][0])
   if segmentation_all:
     gt_mask = GenericMask(segmentation_all, height, width).mask    

   return img, gt_mask, pre_mask_all # gt_mask could be all zero when the ground truth is not given.


### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''
my_data_list_test = DatasetCatalog.get("data_detection_{}".format('test'))
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))

model = model.eval()
img, gt, pre_mask_all = get_prediction_mask(my_data_list_test[20])

from matplotlib.pyplot import figure

#print(pre_mask_all)
figure(num=None, figsize=(15, 15), dpi=80, facecolor='w', edgecolor='k')
plt.imshow(pre_mask_all)
plt.show()
#cv2_imshow(pre_mask_all*255)
# print(np.sum(pre_mask_all<0))
figure(num=None, figsize=(15, 15), dpi=80, facecolor='w', edgecolor='k')
plt.imshow(img)
plt.show()



In [55]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''
import time
print("start genrating train csv")
s = time.time()
my_data_list = DatasetCatalog.get("data_detection_{}".format('train'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  pred_mask = torch.from_numpy(pred_mask).cuda().long()
  #print(pred_mask)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      #print("type ", type(tmp_mask))
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)
print("end train csv, execution time", time.time()-s)



In [ ]:
'''
# Writing the predictions of the test set
'''
s = time.time()
print("start genrating test csv")
my_data_list = DatasetCatalog.get("data_detection_{}".format('test'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  pred_mask= torch.from_numpy(pred_mask).cuda().long()
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()
print("end test csv, execution time", time.time()-s)

## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

### Network

In [26]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg_m = get_cfg()
cfg_m.OUTPUT_DIR = "{}/output2/".format(BASE_DIR)


cfg_m.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
#COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml    350-38.096%
#"COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml" -- 700-54.5%

cfg_m.DATASETS.TRAIN = ("data_detection_train",)
cfg_m.DATASETS.TEST = ()

cfg_m.DATALOADER.NUM_WORKERS = 2

#cfg_m.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg_m.MODEL.WEIGHTS = os.path.join(cfg_m.OUTPUT_DIR, "model_final.pth")
#COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml
cfg_m.SOLVER.IMS_PER_BATCH = 2
cfg_m.SOLVER.BASE_LR = 0.00025  
cfg_m.SOLVER.MAX_ITER = 400
cfg_m.SOLVER.STEPS = [] 

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512


### Training

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
import time
s = time.time()
trainer_mask = DefaultTrainer(cfg_m) 
trainer_mask.resume_or_load(resume=False)
trainer_mask.train()

### Evaluation and Visualization

In [ ]:
'''
# Use COCOEvaluator and build_detection_test_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data_detection_train", cfg_m, False, output_dir="./output/")
#evaluator = COCOEvaluator("data_detection_train", ("bbox", "segm"), False, output_dir="./output/")
val_loader_mask = build_detection_test_loader(cfg_m, "data_detection_train")
print(inference_on_dataset(trainer.model, val_loader_mask, evaluator))

In [27]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg_m.MODEL.WEIGHTS = os.path.join(cfg_m.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg_m.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor_m = DefaultPredictor(cfg_m)


## Visualize Detection

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
from detectron2.utils.visualizer import ColorMode
# load test data
dataset_dicts_test = get_detection_data("test")

for d in [dataset_dicts_test[10]]:    
    im = cv2.imread(d["file_name"])
    outputs = predictor_m(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata= plane_test_meta, 
                   scale=0.4, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])
    # for b in outputs['instances'].pred_boxes:

## Visualize Detction Segmentation Mask


In [ ]:
d = my_data_list_test[40]
im = cv2.imread(d["file_name"])
outputs = predictor_m(im)

masks = outputs['instances'].pred_masks
print(masks.size()) 
pred_boxes = outputs['instances'].pred_boxes
image_w = d['width']
image_h = d['height']
full_mask = np.zeros((image_h,image_w))

for idx, box in enumerate(pred_boxes):
    #box = pred_boxes[idx]
    location = box.cpu().numpy().astype(int)
    x1,y1,x2,y2 = list(location)  
    b_w = x2 - x1
    b_h = y2 - y1
    
  #replace detected box on the original image, seperate value mask for different instance
    ins_mask = masks[idx].cpu().numpy()
    full_mask = full_mask + ins_mask

figure(figsize=(15,15))
plt.imshow(full_mask)
plt.show()



## Evaluation AP

In [ ]:
'''
# Use COCOEvaluator and build_detection_test_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data_detection_train", cfg, False, output_dir="./output/")
#evaluator = COCOEvaluator("data_detection_train", ("bbox", "segm"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

error connecting to /tmp/tmux-0/default (No such file or directory)
